In this notebook will trial sklearn with AI platform

TO DO
- Write training code without worrying about splitting data and submit to AI platform

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/smro/keys/data-sc-activator/data-sc-activator-2cf069890f36.json"

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.oauth2 import service_account
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
! which python

/Users/smro/opt/anaconda3/envs/ml/bin/python


In [3]:
# mkdir for package

In [4]:
!mkdir iris_training

mkdir: iris_training: File exists


In [5]:
!touch ./iris_training/__init__.py

In [6]:
!ls

LICENSE                    iris_training
ai_platform_training.ipynb sklearn_ai_platform.ipynb


In [7]:
# Example code from documentation

In [8]:
%%writefile ./iris_training/train.py

# [START setup]
import datetime
import os
import subprocess
import sys
import pandas as pd
from sklearn import svm
from sklearn.externals import joblib

# Fill in your Cloud Storage bucket name
BUCKET_NAME = '<YOUR_BUCKET_NAME>'
# [END setup]


# [START download-data]
iris_data_filename = 'iris_data.csv'
iris_target_filename = 'iris_target.csv'
data_dir = 'gs://cloud-samples-data/ml-engine/iris'

# gsutil outputs everything to stderr so we need to divert it to stdout.
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_data_filename),
                       iris_data_filename], stderr=sys.stdout)
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_target_filename),
                       iris_target_filename], stderr=sys.stdout)
# [END download-data]


# [START load-into-pandas]
# Load data into pandas, then use `.values` to get NumPy arrays
iris_data = pd.read_csv(iris_data_filename).values
iris_target = pd.read_csv(iris_target_filename).values

# Convert one-column 2D array into 1D array for use with scikit-learn
iris_target = iris_target.reshape((iris_target.size,))
# [END load-into-pandas]


# [START train-and-save-model]
# Train the model
classifier = svm.SVC(gamma='auto', verbose=True)
classifier.fit(iris_data, iris_target)

# Export the classifier to a file
model_filename = 'model.joblib'
joblib.dump(classifier, model_filename)
# [END train-and-save-model]


# [START upload-model]
# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join('gs://', BUCKET_NAME,
    datetime.datetime.now().strftime('iris_%Y%m%d_%H%M%S'), model_filename)
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
    stderr=sys.stdout)
# [END upload-model]

Overwriting ./iris_training/train.py


In [9]:
# Submit job



In [10]:
import time

BUCKET_NAME = "gs://gcp_ai_demo"
JOB_NAME = "iris_job1_{}".format(int(time.time()))

In [11]:
# Submit the training job:
! gcloud ai-platform jobs submit training $JOB_NAME --region us-central1 --runtime-version=2.1 --scale-tier BASIC --python-version=3.7 --staging-bucket $BUCKET_NAME --package-path ./iris_training --module-name iris_training.train


/bin/sh: gcloud: command not found


In [12]:
! gcloud help

/bin/sh: gcloud: command not found


# Load data from Big Query

In [119]:
# Create authenticated client object

key_file = "/home/jupyter/credentials.json"

credentials = service_account.Credentials.from_service_account_file(
        key_file, scopes=["https://www.googleapis.com/auth/cloud-platform"])

client = bigquery.Client(credentials=credentials,
                         project=credentials.project_id)

In [120]:
sql = "SELECT * FROM `data-sc-activator.gcp_ai_demo.diamonds`"

data = pd.read_gbq(sql, project_id=credentials.project_id, credentials=credentials)

Downloading: 100%|██████████| 53940/53940 [00:04<00:00, 11251.89rows/s]


# Write training script

In [151]:
%%writefile ./iris_training/train.py


import pandas as pd
from google.oauth2 import service_account
from google.cloud import storage
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor

# Create authenticated BQ + storage clients
bq_client = storage.Client()
storage_client = storage.Client()

# # Load dataset from BQ
# sql = "SELECT * FROM `data-sc-activator.gcp_ai_demo.diamonds`"
# data = bq_client.query(sql).result().to_dataframe()

# print(data.head())

# y = data['price']
# X = data[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]

# cols_to_encode = ['cut', 'color', 'clarity']
# X_encoded = pd.get_dummies(X[cols_to_encode])
# X_encoded

# X = pd.merge(X, X_encoded, left_index=True, right_index=True)
# X = X.drop(columns=cols_to_encode)

# print(X.columns)

iris = load_iris()
X = iris['data']
y = iris['target']

model = RandomForestRegressor()
model.fit(X, y)

model_name = "basic_model.pkl"

with open(model_name, "wb") as model_file:
    pickle.dump(pipeline, model_file)
    
bucket_name = "gcp_ai_demo"
bucket = storage_client.bucket(bucket_name)
blob.upload_from_filename(model_name)

Writing ./iris_training/train.py


In [152]:
# Submit job to AI Platform training

In [160]:
import time

BUCKET_NAME = "gs://gcp_ai_demo"
JOB_NAME = "iris_job1_{}".format(int(time.time()))

In [161]:
# Submit the training job:
!gcloud ai-platform jobs submit training $JOB_NAME --region us-central1 --runtime-version=2.1 --scale-tier BASIC --python-version=3.7 --staging-bucket $BUCKET_NAME --package-path ./iris_training --module-name iris_training.train


Job [iris_job1_1585329584] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe iris_job1_1585329584

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs iris_job1_1585329584
jobId: iris_job1_1585329584
state: QUEUED


In [140]:
! gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-03-27T16:56:30Z'
etag: VL5XJBMmPhI=
jobId: test_job1_1585328188
state: PREPARING
trainingInput:
  packageUris:
  - gs://gcp_ai_demo/test_job1_1585328188/01ee8964d1fa1010bf07923521ee4623cad578cd4860790c696847777c53d95b/diamond_training-0.0.0.tar.gz
  pythonModule: diamond_training.train
  pythonVersion: '3.7'
  region: us-central1
  runtimeVersion: '2.1'
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/test_job1_1585328188?project=data-sc-activator

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2Ftest_job1_1585328188&project=data-sc-activator


In [126]:
! echo $BUCKET_NAME

gs://gcp_ai_demo


In [127]:
! echo $JOB_NAME

test_job1_1585327966


# Serve the model

In [128]:
MODEL_NAME = "diamond_test_model"
VERSION_NAME = "diamond_test_model_2"

In [129]:
!gcloud ai-platform models create $MODEL_NAME --regions us-central1

ERROR: (gcloud.ai-platform.models.create) Resource in project [data-sc-activator] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [130]:
!gcloud ml-engine versions create $VERSION_NAME --model=$MODEL_NAME --framework=scikit-learn --origin=gs://$BUCKET_NAME/

ERROR: (gcloud.ml-engine.versions.create) FAILED_PRECONDITION: Field: version.deployment_uri Error: The provided GCS prefix [gs://gs://gcp_ai_demo/] cannot be read by service account service-1080702431163@cloud-ml.google.com.iam.gserviceaccount.com.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The provided GCS prefix [gs://gs://gcp_ai_demo/] cannot be read by
      service account service-1080702431163@cloud-ml.google.com.iam.gserviceaccount.com.
    field: version.deployment_uri


In [131]:
# Test alternative method to get bq data into pandas

In [141]:
from google.cloud import bigquery


sql = "SELECT * FROM `data-sc-activator.gcp_ai_demo.diamonds`"

bq_client = bigquery.Client(credentials=credentials,
                         project=credentials.project_id)

# |
    
df = bq_client.query(sql).result().to_dataframe()
df
# dataframe = (
#     bq_client.query(sql)
#     .result()
#     .to_dataframe(bqstorage_client=bq_client))

,carat,cut,color,clarity,depth,table,price,x,y,z
0,2.00,Good,D,SI2,63.8,57.0,10528,7.93,7.88,5.04
1,2.00,Premium,D,SI2,59.3,62.0,12576,8.12,8.06,4.80
2,2.00,Premium,D,SI2,61.9,59.0,13098,8.02,8.00,4.98
3,2.00,Premium,D,SI2,61.0,58.0,14406,8.06,7.97,4.89
4,2.00,Fair,D,SI1,64.8,58.0,15540,7.98,7.89,5.14
...,...,...,...,...,...,...,...,...,...,...
53935,1.56,Premium,J,VS1,61.1,59.0,8476,7.52,7.49,4.58
53936,1.56,Ideal,J,VS1,62.5,56.0,8763,7.38,7.46,4.64
53937,1.56,Ideal,J,VS1,61.9,55.0,9404,7.44,7.46,4.61
53938,1.81,Fair,J,I1,68.0,57.0,5859,7.43,7.39,5.04


In [145]:
from sklearn.datasets import load_iris

In [146]:
iris = load_iris()

In [147]:
iris = load_iris()
X = iris['data']
y = iris['target']

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [157]:
iris = load_iris()
X = iris['data']
y = iris['target']

model = RandomForestRegressor()
model.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [158]:
model.score(X, y)

0.9925